# Ask about the HPC compendium
In this RAG example, we will use the [HPC compendium](https://compendium.hpc.tu-dresden.de/) which is licensed under [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/) by TU Dresden ZIH. 

In [1]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import os

In [2]:
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai_like import OpenAILike

# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# ollama
Settings.llm = OpenAILike(model="meta-llama/Llama-3.3-70B-Instruct", request_timeout=120.0, api_base="https://llm.scads.ai/v1", api_key=os.environ.get('SCADSAI_API_KEY'))

In [3]:
class Chat():
    def __init__(self):
        self.former_folder = None
        
    def load(self, folder):
        if self.former_folder == folder:
            return self.count_documents()
        self.former_folder = folder
        
        documents = SimpleDirectoryReader(folder, required_exts=[".md"], recursive=True).load_data()
        index = VectorStoreIndex.from_documents(documents)
        self._query_engine = index.as_query_engine()
        self._documents = documents
        return self.count_documents()

    def count_documents(self):
        return len(np.unique([d.metadata["file_name"] for d in self._documents]))
        
    def query(self, question):
        return self._query_engine.query(question)

In [4]:
# Create user interface
docs_input = widgets.Text(value="hpc-compendium/doc.zih.tu-dresden.de/docs/", placeholder="Enter a directory here")
load_button = widgets.Button(description="Load")

output_label = widgets.HTML(value="")

text_input = widgets.Text(placeholder="Enter a question here")
submit_button = widgets.Button(description="Submit")

chat = Chat()

def on_load(e=None):    
    number_of_documents = chat.load(docs_input.value)
    output_label.value = f"""
    <div style='text-align:left; color: darkgrey; font-size: 20px'>{number_of_documents} documents loaded.</div>
    """
    if number_of_documents > 0:
        submit_button.disabled = False

def on_submit(e):
    question = text_input.value
    if len(question) == 0:
        return
    text_input.value = ""

    # submit prompt to LLM
    answer = chat.query(question)

    # Append question and answer to the existing HTML content
    output_label.value += f"""
    <div style='text-align:right; color: blue; font-size: 20px'>{question}</div>
    <div style='text-align:left; color: darkgreen; font-size: 20px'>{answer}</div>
    """

# Attach the event handler to the text field and the button
docs_input.continuous_update = False
docs_input.observe(on_load)
load_button.on_click(on_load)

text_input.continuous_update = False
text_input.observe(on_submit)
submit_button.on_click(on_submit)
submit_button.disabled = True

print("loading...")
on_load()
print("done")

# Arrange the widgets for display
display(widgets.HBox([docs_input, load_button]), output_label, widgets.HBox([text_input, submit_button]))

loading...
done


HTML(value="\n    <div style='text-align:left; color: darkgrey; font-size: 20px'>127 documents loaded.</div>\n…